In [32]:
#Beginnings of systematic iterative strategy testing. The backtrader cerebro run is contained within a function with
#strictly defined inputs and outputs. Currently, cerebro crashes sometimes but not always

#TODO: 
#make another getInputs function that will pull random, but valid dates to get data over. 
#Build the above on top of a getInput function that does not cite a SYM.txt file
#Try using longer-term data sets

import backtrader as bt
import pandas as pd
import yfinance as yf



In [33]:

from typing import Dict, List
from functools import reduce
from pandas import DataFrame
# --------------------------------
import talib.abstract as tab
from typing import Dict, List
from pandas import DataFrame, DatetimeIndex, merge
# --------------------------------
import talib as ta
import numpy, random, time, os
import datetime as dt
#from dateutil.parser import parse


In [34]:
###BT STRAT CLASSES
class Scalp(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=10)
        self.emaclose=bt.indicators.EMA(self.data.close, period=10)
        self.emalow=bt.indicators.EMA(self.data.low, period=10)
        
        #creat stochastic momentum indicators 
        self.stoch=bt.talib.STOCHF(self.data.high, self.data.low, self.data.close, \
                                   fastk_period=14, slowk_period=4, slowd_period=4)
        self.fastk=self.stoch.fastk
        self.fastd=self.stoch.fastd
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.data.open < self.emalow and \
            self.adx > 30 and \
            self.fastk < 30 and \
            self.fastd < 30:
                self.buy()
                
                
        else:
            if self.data.open >= self.emahigh and \
            self.fastk > 70 and \
            self.fastd > 70:
                self.sell()

In [35]:
###FUNCTIONS

def writeInputOutput(Strategy,Input,Output,CSVfilename):
#The column order in the CSV is currently: 
#Timestamp; Strategy; Symbol; StartDate; EndDate; Interval; Set; PercentIncrease; Ntrades
#Strategy is the strategy name string, input/output are dicts, CSVfilename= BTArchive1.csv
#Assumes a such a file already exists, copypaste the above line into a txt, call it XXXX.csv

    if not CSVfilename in os.listdir(): #don't write a new file. 
        print(CSVfilename+' not found.')
        return
    with open(CSVfilename,'a') as CSV: #append mode
        Timestamp = str(int(time.time())) #"ID number"
        time.sleep(1)
        PercentIncreaseStr = '{:.2f}'.format(Output['PercentIncrease'])
        
        CSV.write(Timestamp+';'+Strategy+';'+Input['SYM']+';'+Input['StartDate']+';'+Input['EndDate']+';'+
                 Input['Interval']+';'+Input['Set']+';'+PercentIncreaseStr+';'+str(Output['Ntrades'])+"\n")
        CSV.close()
        
def getInputs(N,SymListTxt,StartDate,EndDate,Interval):
    #N=number of symbols to pull, SymListText = "data set" to pull them from (must end with SYM.txt, 
        #i.e. BigCapSYM.txt) 
    #Interval = bar interval (same as yf, '1m', 30m, 1d, 1mo, et),
    #Dates as a string ('2020-04-06')
    
    
    SYMS = [False]*N
    Inputs = []
    for i in range(N):
        while True: #Keep going until you get a valid YDF
            #choose a symbol, no repeats
            f = open(SymListTxt)
            flines=f.readlines()
            while True:
                SYM = random.choice(flines).rstrip("\n")
                if not SYM in SYMS:
                    break
            f.close()
            #download
            YDF = yf.download([SYM],start=StartDate,end=EndDate,interval=Interval)
            if not YDF.shape[0]==0: #No data was downloaded, yf will describe what went wrong
                SYMS[i]=SYM
                break
        Inputs.append({'YDF':YDF,'SYM':SYM,'StartDate':StartDate,
                       'EndDate':EndDate,'Interval':Interval,
                      'Set':SymListTxt.rstrip("SYM.txt")})
        #dt.datetime.strptime(StartDate,'%Y-%m-%d')
    return Inputs #returns list of dicts, dicts containing the YDF and its important qualifiers

def testInput(Input,StratName,WriteFile='BTArchive1.csv'):
    #feed dataframe to cerbro
    datacere = bt.feeds.PandasData(dataname=Input['YDF'])
    
    #create backtrader
    cerebro = bt.Cerebro()
    #Find strategy. I could do this with eval(StartName), but that's not safe
    if StratName == 'Scalp':
        cerebro.addstrategy(Scalp)
    else:
        print('StratName not Found')
        return

    #set up cerebro
    StartCash = 100000 #I would like this number for later
    cerebro.broker.setcash(StartCash)
    cerebro.broker.setcommission(commission=0.0)
    cerebro.addsizer(bt.sizers.AllInSizer)
    #All In Sizer always uses maximum cash possible, meaning we can divide the profit by the starting cash later to
    #get a normalized percent increase
    cerebro.adddata(datacere)
    #Analyzers. I am not sure how many of these lines are necessary
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
    cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")
    cerebro.addanalyzer(bt.analyzers.Returns, _name="ret")
    cerebro.addanalyzer(bt.analyzers.PeriodStats, _name="pstat")
    cerebro.addanalyzer(bt.analyzers.Transactions, _name="trans")
    
    #Run
    try:
        result = cerebro.run()
        analysis = result[0].analyzers.ta.get_analysis()
    except IndexError:
        print('Error in Cerebro')
        return False
    #Get the output
    Gross = cerebro.broker.getvalue()
    PercentIncrease= (Gross-StartCash)/StartCash*100
    Ntrades = analysis['total']['total']
    Output = {'PercentIncrease':PercentIncrease,'Ntrades':Ntrades}
    
    if bool(WriteFile):
        writeInputOutput(StratName,Input,Output,WriteFile)
    print('Used {} on {} for {:.2f}% with {:d} trades'.format(StratName,Input['SYM'],Output['PercentIncrease'],
                                                           Output['Ntrades']))
    return Output
    #Output is also a dictionary. May add other things here later
    #TODO: incporate writing to a CSV or Excel directly in this function


    
            
            

In [36]:
#Get Inputs
Inputs = getInputs(5,'AllSYM.txt','2020-04-03','2020-04-04','1m')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- GRNVU: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [37]:
#Test Inputs
Outputs = [testInput(I,'Scalp') for I in Inputs] 

Used Scalp on BABA for 0.00% with 1 trades
Used Scalp on EWZS for 0.00% with 0 trades
Used Scalp on VICI for 1.62% with 4 trades
Used Scalp on ACV for 0.00% with 0 trades
Used Scalp on STAG for 3.58% with 4 trades
